In [1]:
import nltk

In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

In [3]:
sid = SentimentIntensityAnalyzer()

In [4]:
df = pd.read_json('google_evandro.json')
df.replace(r'\n', '', regex=True, inplace=True)

In [5]:
df

,Title,Date,Contributor,Content,Link,Role
0,Google AI Blog: Permutation-Invariant Neural N...,2021-11-18,"[David Ha, Yujin Tang]","Posted by David Ha, Staff Research Scientist a...",https://ai.googleblog.com/2021/11/permutation-...,"[Staff Research Scientist, Research Software E..."
1,Google AI Blog: Making Better Future Predictio...,2021-11-11,"[Dave Epstein, Chen Sun]","Posted by Dave Epstein, Student Researcher and...",https://ai.googleblog.com/2021/11/making-bette...,"[Student Researcher, Staff Research Scientist]"
2,Google AI Blog: An Open Source Vibrotactile Ha...,2021-11-12,[Artem Dementyev],"Posted by Artem Dementyev, Hardware Engineer, ...",https://ai.googleblog.com/2021/11/an-open-sour...,[Hardware Engineer]
3,Google AI Blog: MetNet-2: Deep Learning for 12...,2021-11-15,"[Nal Kalchbrenner, Lasse Espeholt]","Posted by Nal Kalchbrenner and Lasse Espeholt,...",https://ai.googleblog.com/2021/11/metnet-2-dee...,
4,Google AI Blog: RLiable: Towards Reliable Eval...,2021-11-17,"[Rishabh Agarwal, Pablo Samuel Castro]","Posted by Rishabh Agarwal, Research Scientist ...",https://ai.googleblog.com/2021/11/rliable-towa...,"[Research Scientist, Staff Software Engineer]"
...,...,...,...,...,...,...
1004,Google AI Blog: First Robots,2006-03-22,"[Sumit Agarwal, & Michael Stoppelman]","Posted by Sumit Agarwal, Maryam Kamvar, & Mic...",https://ai.googleblog.com/2006/03/first-robots...,[Maryam Kamvar]
1005,Google AI Blog: See you at CHI,2006-04-23,[Rick Boardman],"Posted by Rick Boardman, User Experience Resea...",https://ai.googleblog.com/2006/04/see-you-at-c...,[User Experience Researcher]
1006,Google AI Blog: Our conference on automated te...,2006-04-27,[Allen Hutchison],"Posted by Allen Hutchison, Engineering Manager...",https://ai.googleblog.com/2006/04/our-conferen...,[Engineering Manager]
1007,Google AI Blog: Statistical machine translatio...,2006-04-28,[Franz Och],"Posted by Franz Och, Research ScientistBecause...",https://ai.googleblog.com/2006/04/statistical-...,[Research Scientist]


In [6]:
df.isnull().sum()

Title          0
Date           0
Contributor    0
Content        0
Link           0
Role           0
dtype: int64

In [7]:
sid.polarity_scores(df.iloc[0]['Content'])

{'neg': 0.015, 'neu': 0.924, 'pos': 0.061, 'compound': 0.9969}

In [8]:
df.Content[0]

"Posted by David Ha, Staff Research Scientist and Yujin Tang, Research Software Engineer, Google Research, Tokyo“The brain is able to use information coming from the skin as if it were coming from the eyes. We don’t see with the eyes or hear with the ears, these are just the receptors, seeing and hearing in fact goes on in the brain.”— Paul Bach-y-Rita1People have the amazing ability to use one sensory modality (e.g., touch) to supply environmental information normally gathered by another sense (e.g., vision). This adaptive ability, called sensory substitution, is a phenomenon well-known to neuroscience. While difficult adaptations — such as adjusting to seeing things upside-down, learning to ride a “backwards” bicycle, or learning to “see” by interpreting visual information emitted from a grid of electrodes placed on one’s tongue — require anywhere from weeks, months or even years to attain mastery, people are able to eventually adjust to sensory substitutions.Examples of Sensory Subs

In [9]:
df['Scores'] = df['Content'].apply(lambda content: sid.polarity_scores(content))

In [10]:
df

,Title,Date,Contributor,Content,Link,Role,Scores
0,Google AI Blog: Permutation-Invariant Neural N...,2021-11-18,"[David Ha, Yujin Tang]","Posted by David Ha, Staff Research Scientist a...",https://ai.googleblog.com/2021/11/permutation-...,"[Staff Research Scientist, Research Software E...","{'neg': 0.015, 'neu': 0.924, 'pos': 0.061, 'co..."
1,Google AI Blog: Making Better Future Predictio...,2021-11-11,"[Dave Epstein, Chen Sun]","Posted by Dave Epstein, Student Researcher and...",https://ai.googleblog.com/2021/11/making-bette...,"[Student Researcher, Staff Research Scientist]","{'neg': 0.009, 'neu': 0.917, 'pos': 0.074, 'co..."
2,Google AI Blog: An Open Source Vibrotactile Ha...,2021-11-12,[Artem Dementyev],"Posted by Artem Dementyev, Hardware Engineer, ...",https://ai.googleblog.com/2021/11/an-open-sour...,[Hardware Engineer],"{'neg': 0.02, 'neu': 0.903, 'pos': 0.076, 'com..."
3,Google AI Blog: MetNet-2: Deep Learning for 12...,2021-11-15,"[Nal Kalchbrenner, Lasse Espeholt]","Posted by Nal Kalchbrenner and Lasse Espeholt,...",https://ai.googleblog.com/2021/11/metnet-2-dee...,,"{'neg': 0.019, 'neu': 0.91, 'pos': 0.071, 'com..."
4,Google AI Blog: RLiable: Towards Reliable Eval...,2021-11-17,"[Rishabh Agarwal, Pablo Samuel Castro]","Posted by Rishabh Agarwal, Research Scientist ...",https://ai.googleblog.com/2021/11/rliable-towa...,"[Research Scientist, Staff Software Engineer]","{'neg': 0.025, 'neu': 0.868, 'pos': 0.107, 'co..."
...,...,...,...,...,...,...,...
1004,Google AI Blog: First Robots,2006-03-22,"[Sumit Agarwal, & Michael Stoppelman]","Posted by Sumit Agarwal, Maryam Kamvar, & Mic...",https://ai.googleblog.com/2006/03/first-robots...,[Maryam Kamvar],"{'neg': 0.018, 'neu': 0.784, 'pos': 0.198, 'co..."
1005,Google AI Blog: See you at CHI,2006-04-23,[Rick Boardman],"Posted by Rick Boardman, User Experience Resea...",https://ai.googleblog.com/2006/04/see-you-at-c...,[User Experience Researcher],"{'neg': 0.01, 'neu': 0.862, 'pos': 0.128, 'com..."
1006,Google AI Blog: Our conference on automated te...,2006-04-27,[Allen Hutchison],"Posted by Allen Hutchison, Engineering Manager...",https://ai.googleblog.com/2006/04/our-conferen...,[Engineering Manager],"{'neg': 0.025, 'neu': 0.803, 'pos': 0.172, 'co..."
1007,Google AI Blog: Statistical machine translatio...,2006-04-28,[Franz Och],"Posted by Franz Och, Research ScientistBecause...",https://ai.googleblog.com/2006/04/statistical-...,[Research Scientist],"{'neg': 0.018, 'neu': 0.906, 'pos': 0.075, 'co..."


In [11]:
df['Compound'] = df['Scores'].apply(lambda d: d['compound'])

In [12]:
df['Overall Score'] = df['Compound'].apply(lambda score: 'POS' if score >=0 else 'NEG')

In [13]:
df[df['Overall Score'] == 'NEG']

,Title,Date,Contributor,Content,Link,Role,Scores,Compound,Overall Score
44,Google AI Blog: Demonstrating the Fundamentals...,2021-08-11,"[Jimmy Chen, Matt McEwen]","Posted by Jimmy Chen, Quantum Research Scienti...",https://ai.googleblog.com/2021/08/demonstratin...,"[Quantum Research Scientist, Student Researcher]","{'neg': 0.117, 'neu': 0.81, 'pos': 0.073, 'com...",-0.9969,NEG
76,Google AI Blog: Learning to Manipulate Deforma...,2021-05-14,"[Daniel Seita, Andy Zeng]","Posted by Daniel Seita, Research Intern and An...",https://ai.googleblog.com/2021/05/learning-to-...,"[Research Intern, Research Scientist]","{'neg': 0.07, 'neu': 0.868, 'pos': 0.062, 'com...",-0.9216,NEG
85,Google AI Blog: Do Wide and Deep Networks Lear...,2021-05-04,[Thao Nguyen],"Posted by Thao Nguyen, AI Resident, Google Res...",https://ai.googleblog.com/2021/05/do-wide-and-...,[AI Resident],"{'neg': 0.062, 'neu': 0.896, 'pos': 0.042, 'co...",-0.9849,NEG
187,Google AI Blog: Machine Learning-based Damage ...,2020-06-16,"[Joseph Xu, Pranav Khaitan]","Posted by Joseph Xu, Senior Software Engineer ...",https://ai.googleblog.com/2020/06/machine-lear...,"[Senior Software Engineer, Engineering Lead]","{'neg': 0.134, 'neu': 0.755, 'pos': 0.111, 'co...",-0.9950,NEG
210,Google AI Blog: Optimizing Multiple Loss Funct...,2020-04-27,[Alexey Dosovitskiy],"Posted by Alexey Dosovitskiy, Research Scienti...",https://ai.googleblog.com/2020/04/optimizing-m...,[Research Scientist],"{'neg': 0.078, 'neu': 0.869, 'pos': 0.054, 'co...",-0.9821,NEG
292,Google AI Blog: Bi-Tempered Logistic Loss for ...,2019-08-26,"[Ehsan Amid, Rohan Anil]","Posted by Ehsan Amid, Student Researcher and R...",https://ai.googleblog.com/2019/08/bi-tempered-...,"[Student Researcher, Software Engineer]","{'neg': 0.127, 'neu': 0.793, 'pos': 0.08, 'com...",-0.9953,NEG
299,Google AI Blog: Robust Neural Machine Translation,2019-07-29,[Yong Cheng],"Posted by Yong Cheng, Software Engineer, Googl...",https://ai.googleblog.com/2019/07/robust-neura...,[Software Engineer],"{'neg': 0.116, 'neu': 0.78, 'pos': 0.104, 'com...",-0.9657,NEG
379,Google AI Blog: Applying Deep Learning to Meta...,2018-10-12,"[Martin Stumpe, Craig Mermel]","Posted by Martin Stumpe, Technical Lead and Cr...",https://ai.googleblog.com/2018/10/applying-dee...,"[Technical Lead, Product Manager]","{'neg': 0.11, 'neu': 0.787, 'pos': 0.104, 'com...",-0.9624,NEG
390,Google AI Blog: Introducing the Unrestricted A...,2018-09-13,"[Tom B. Brown, Catherine Olsson]","Posted by Tom B. Brown and Catherine Olsson, R...",https://ai.googleblog.com/2018/09/introducing-...,Research Engineers,"{'neg': 0.123, 'neu': 0.769, 'pos': 0.108, 'co...",-0.9592,NEG
424,Google AI Blog: An Augmented Reality Microscop...,2018-04-16,"[Martin Stumpe, Craig Mermel]","Posted by Martin Stumpe, Technical Lead and Cr...",https://ai.googleblog.com/2018/04/an-augmented...,"[Technical Lead, Product Manager]","{'neg': 0.058, 'neu': 0.874, 'pos': 0.068, 'co...",-0.1548,NEG


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
cv = CountVectorizer(max_df=.9, min_df=.1, stop_words= 'english')

In [16]:
dtm = cv.fit_transform(df['Content'])

In [17]:
dtm

<1009x590 sparse matrix of type '<class 'numpy.int64'>'
	with 115286 stored elements in Compressed Sparse Row format>

In [18]:
from sklearn.decomposition import LatentDirichletAllocation

In [19]:
LDA = LatentDirichletAllocation(n_components=10)

In [20]:
LDA.fit(dtm)

LatentDirichletAllocation()

In [21]:
len(cv.get_feature_names()) #Mesmo tamanho do dtm, ou seja, são mil e poucas dimensoes (documentos) por 590 linhas

C:\Users\mvmoraes\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


590

In [22]:
cv.get_feature_names()[100]

'conference'

In [23]:
len(LDA.components_) #Retorna o numero de tópicos

10

In [24]:
first_topic = LDA.components_[0]

In [25]:
type(first_topic)

numpy.ndarray

In [26]:
first_topic.argsort() #retorna um array com o numero dos indices dos valores sorteados em ordem do menor pro maior

#ex: [10, 200, 1] retornaria [2, 0, 1]

array([507, 169, 279, 222, 524, 353, 422, 113, 350,  53, 301, 527, 223,
        45, 538, 144, 171, 546, 562, 580, 184, 533, 108, 496, 554, 125,
       358, 430, 203, 499, 172, 264, 490, 486, 326, 468, 187,  69, 569,
        52, 417, 150, 406, 469,  41, 374, 489, 403, 536, 269, 581, 514,
       364, 148, 337,  75, 266, 457, 318, 412, 361, 272, 140, 453, 319,
       183, 428, 283, 563, 446, 559, 320, 134,  93, 239, 515, 488, 188,
       305,  95, 583, 306, 577,  60,  96, 377, 162, 296, 418, 473, 190,
       139, 315,  94, 416, 547, 541,  23,  99, 164, 154, 103, 286, 120,
       414, 173, 304, 383, 481, 119, 170, 271, 349, 566, 191, 167,  74,
       525, 467, 299,  91, 578,  10, 384, 574,  85, 576, 357,  62, 366,
       390, 189, 400, 336, 152, 158, 205, 309, 352, 226, 386, 260, 516,
       256, 558, 439, 497, 314, 265, 186, 224, 308, 229, 105, 396, 137,
        35, 181, 401, 461, 330, 407, 208,  59, 548, 419, 431, 550, 505,
       584, 579, 402, 460, 101, 323, 388, 117, 136, 435,  13, 25

In [27]:
top_ten = first_topic.argsort()[-10:]

In [28]:
for index in top_ten:
    print(cv.get_feature_names_out()[index])

visual
models
vision
objects
learning
training
object
dataset
images
image


In [29]:
import sys

orig_stdout = sys.stdout
f = open(
    'df.txt', 'w'
)
sys.stdout = f


for index, topic in enumerate(LDA.components_):
    dataframe = pd.DataFrame(columns=[f'THE TOP 15 WORDS FOR TOPIC #{index}'], data=[cv.get_feature_names_out()[index] for index in topic.argsort()[-15:]])
    print(dataframe)

sys.stdout = orig_stdout
f.close()

In [30]:
df2 = pd.read_table('df.txt', header=None)
import numpy as np
arr = np.array(df2)
dataframe = pd.DataFrame(arr.reshape((10, 16))).T
headers = dataframe.iloc[0]
new_df = pd.DataFrame(data=dataframe.values[1:], columns=headers)
new_df

,THE TOP 15 WORDS FOR TOPIC #0,THE TOP 15 WORDS FOR TOPIC #1,THE TOP 15 WORDS FOR TOPIC #2,THE TOP 15 WORDS FOR TOPIC #3,THE TOP 15 WORDS FOR TOPIC #4,THE TOP 15 WORDS FOR TOPIC #5,THE TOP 15 WORDS FOR TOPIC #6,THE TOP 15 WORDS FOR TOPIC #7,THE TOP 15 WORDS FOR TOPIC #8,THE TOP 15 WORDS FOR TOPIC #9
0,0 labels,0 accuracy,0 machine,0 used,0 work,0 reinforcement,0 different,0 cloud,0 page,0 technology
1,1 supervised,1 new,1 systems,1 user,1 based,1 algorithms,1 left,1 information,1 based,1 course
2,2 datasets,2 time,2 example,2 mobile,2 performance,2 david,2 layer,2 ve,2 datasets,2 work
3,3 detection,3 state,3 data,3 videos,3 different,3 optimization,3 using,3 help,3 large,3 online
4,4 model,4 neural,4 natural,4 input,4 ml,4 models,4 like,4 time,4 knowledge,4 faculty
5,5 visual,5 architecture,5 speech,5 quality,5 used,5 li,5 high,5 world,5 information,5 systems
6,6 models,6 using,6 english,6 device,6 example,6 networks,6 example,6 use,6 research,6 year
7,7 vision,7 learning,7 models,7 pixel,7 real,7 chen,7 layers,7 make,7 video,7 new
8,8 objects,8 task,8 model,8 using,8 using,8 research,8 right,8 like,8 algorithms,8 computing
9,9 learning,9 network,9 word,9 time,9 dataset,9 tensorflow,9 resolution,9 people,9 dataset,9 program


In [31]:
topic_results = LDA.transform(dtm)

In [32]:
topic_results[0].argmax()

1

In [36]:
df['Topic'] = topic_results.argmax(axis=1)

,Topic
7,189
9,150
4,130
1,125
3,95
2,78
5,69
0,69
8,64
6,40
